In [1]:
#new epoching file

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import label, binary_dilation, binary_erosion
from scipy.ndimage import find_objects
import os.path as op
import mne
import random

In [3]:
detected_SWRs = {#"Patient3_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient3_part1_SWRs.csv",
            "Patient5_part1L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient5_part1L_SWRs.csv" ,
            "Patient5_part1R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient5_part1R_SWRs.csv",
            "Patient5_part2L" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient5_part2L_SWRs.csv",
            "Patient5_part2R" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient5_part2R_SWRs.csv",
            "Patient6" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient6_SWRs.csv",
            "Patient8" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient8_SWRs.csv",
            #"Patient9" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient9_SWRs.csv",
            "Patient10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient10_SWRs.csv",
            "Patient11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient11_part1_SWRs.csv",
            'Patient11_part2' :"/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient11_part2_SWRs.csv",
            #"Patient12_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient12_part1_SWRs.csv",
            "Patient13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient13_SWRs.csv",
            "Patient14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient14_part1_SWRs.csv",
            "Patient14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient14_part2_SWRs.csv",
            "Patient14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient14_part1_SWRs.csv",
            "Patient14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient14_part2_SWRs.csv",
            #"Patient15" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient15_SWRs.csv",
            "Patient16" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient16_SWRs.csv",
            "Patient17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient17_SWRs.csv",
            "Patient21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient21_SWRs.csv",
            "Patient22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient22_SWRs.csv",
            "Patient23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient23_SWRs.csv",
            "Patient25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/SWR_times/Patient25_SWRs.csv"
            }

In [4]:
hippocampus_electrodes = {
   #'Patient3_part1': [],
    'Patient5_part1L': ['EEG026', 'EEG028', 'EEG032', 'EEG034', 'EEG057', 'EEG059'],
    'Patient5_part1R': ['EEG001', 'EEG003','EEG026', 'EEG028', 'EEG032', 'EEG034'],
    'Patient5_part2L': ['EEG026', 'EEG028', 'EEG032', 'EEG034', 'EEG057', 'EEG059'],
    'Patient5_part2R': ['EEG001', 'EEG003','EEG026', 'EEG028', 'EEG032', 'EEG034'],
    'Patient6': ['EEG001', 'EEG002','EEG003', 'EEG004', 'EEG005', 'EEG006', 'EEG007', 'EEG008'],
   'Patient8': ['EEG026', 'EEG027', 'EEG028'],
    #'Patient9': [],
    'Patient10': ['EEG013'], 
    'Patient11_part1': ['EEG043', 'EEG044', 'EEG045', 'EEG046'],
    'Patient11_part2': ['EEG043', 'EEG044', 'EEG045', 'EEG046'],
   # 'Patient12_part1': [],
    #'Patient12_part2': [],
    'Patient13': ['EEG036'] ,
    'Patient14_part1_left': ['EEG001'],
   'Patient14_part2_left': ['EEG001'],
   'Patient14_part1_right': ['EEG041', 'EEG042', 'EEG043'],
    'Patient14_part2_right': ['EEG041', 'EEG042', 'EEG043'],
  #'Patient15': [],
   'Patient16': ['EEG003'],
    'Patient17': ['EEG003', 'EEG004', 'EEG005', 'EEG006'],
    #'Patient19': [],
    'Patient21': ['EEG002','EEG003', 'EEG004', 'EEG005'],
   'Patient22':  ['EEG001', 'EEG002', 'EEG003','EEG017', 'EEG018', 'EEG019', 'EEG020', 'EEG021', 'EEG022'], #[]
   'Patient23': ['EEG057', 'EEG058', 'EEG059'],
    'Patient25': ['EEG049', 'EEG050']
}

In [5]:
recording_durations = { #number of expected SWrs (based on paper suggesting normal range is 10-40per min)
    'Patient3_part1': 340, #55 - 225
    'Patient5_part1L': 358, #60- 240
    'Patient5_part1R': 313, #52 - 210
    'Patient5_part2L': 293, #48-195
    'Patient5_part2R': 257, #43 - 141
    'Patient6': 414, #70 - 280
    'Patient8': 575, #95 - 383
    'Patient9': 356, #60 - 237
    'Patient10': 627, #100 - 400
    'Patient11_part1': 431, #68 - 275
    'Patient11_part2': 352, #60 - 240
    'Patient12_part1': 409.02,#70 - 280
    'Patient12_part2':656.88 ,#110 - 440
    'Patient13': 243, #40 - 160
    'Patient14_part1_left': 452, #75 - 300
    'Patient14_part2_left': 497,#80 - 330
    'Patient14_part1_right': 452, #75 - 300
    'Patient14_part2_right': 497,
    'Patient15': 190,#30 - 125
    'Patient16': 181,#30 - 120
    'Patient17': 188,#30 - 120
    'Patient19': 1159,#190 - 772
    'Patient21': 353,# 60 - 240
    'Patient22': 614,#100 - 400
    'Patient23': 758, #125 -505
    'Patient25': 256# 40 - 170
}

In [9]:
#import raw files
raw_files = {#"Patient03" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/FM_patient3/FM_9to16_part1_raw_tsss.fif",
            'Patient5_part1L' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_LHippo_001_tsss.fif" ,
            'Patient5_part1R' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_Rhippo_raw_001_tsss.fif",
            'Patient5_part2L' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_Lhippo_raw_002_tsss.fif",
            'Patient5_part2R' : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_RHippo_raw_002_tsss.fif",
            "Patient6" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZM_patient6/ZM_raw_001_tsss.fif",
            "Patient8" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/XX_patient8/XX_raw_001_tsss.fif",
           # "Patient9" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/TY_patient9/TY_raw_001_tsss.fif",
            "Patient10" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/NY_patient10/NY_raw_001_tsss.fif",
            "Patient11_part1" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZJ_patient11/ZJ_raw_001_tsss.fif",
            "Patient11_part2" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZJ_patient11/ZJ_raw_002_tsss.fif",
            #"Patient12" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/JC_patient12/JC_raw_001_tsss.fif",
            "Patient13" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZB_patient13/ZB_raw_004_tsss.fif",
            "Patient14_part1_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_001_tsss.fif",
            "Patient14_part2_left" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_003_tsss.fif",
            "Patient14_part1_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_001_tsss.fif",
            "Patient14_part2_right" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/CL_patient14/CL_raw_003_tsss.fif",
            #"Patient15" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZX_patient15/ZX_EP_rest_001_tsss.fif",
            "Patient16" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/RB_patient16/RB_EP_SEEG_rest_001_tsss.fif",
            "Patient17" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/LJM_patient17/LJM_EP_SEEG_rest_001_tsss.fif",
            "Patient21" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/HH_patient21/HH_SEEG_001_tsss.fif",
            "Patient22" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/YY_patient22/YY_SEEG_raw_001_tsss.fif",
            "Patient23" : "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/ZQ_patient23/ZQ_EP_SEEG_001_tsss.fif",
            "Patient25":  "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WL_patient25/WL_raw_rest_001_tsss.fif"
            }




In [7]:
#parametetes and save paths
eeg_fs = 1000
min_gap = 50
no_of_epochs = 30

region_output_path = "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/hipp_SWR_regions_new/mag"
epochs_save_path = "/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_epochs/mag"

In [10]:
for patient_id, electrodes in hippocampus_electrodes.items():
    df = pd.read_csv(detected_SWRs[patient_id])
    df = df[df['Channel'].isin(electrodes)]
    if df.empty:
        print(f"No hippocampal channels found for {patient_id}")
        continue
    SWR_times = (df['rippleTime'].apply(lambda x: eval(x, {'array': np.array}))).tolist()
    #SWR_times = df['rippleTime'].apply(eval).apply(lambda lst: [np.array(seg) for seg in lst]).tolist()
    duration = recording_durations[patient_id]
    eeg_no_samples = int(duration * eeg_fs)
    no_channels = len(df)
    SWR_array = np.zeros((no_channels, eeg_no_samples), dtype=int)
    for channel, times in enumerate(SWR_times):
        if not times or all(len(t) == 0 for t in times):
            ripple_time_value = df.iloc[channel]['rippleTime'] 
            print(f"Skipping empty SWR_times for channel index {channel} in patient {patient_id}")
            continue
        channel_SWR_times = np.hstack(times)
        channel_SWR_idx = np.round((channel_SWR_times * eeg_fs), 0).astype(int)
        SWR_array[channel, channel_SWR_idx] = 1

    SWR_sum = np.sum(SWR_array, axis = 0)

    #1. get regions where SWRs were detected
    valid_regions = (SWR_sum >= 1)
    #dialate and erode to merge neighbouring regions
    min_duration_samples = int(10)
    structure_element = np.ones(min_duration_samples)
    valid_regions = binary_dilation(valid_regions, structure=structure_element)
    valid_regions = binary_erosion(valid_regions, structure=structure_element)
    #need to label connected valid regions
    labeled_regions, num_regions = label(valid_regions)
    slices = find_objects(labeled_regions)
    #manually compoute region properties
    region_peaks = []
    for region in slices:
        start = region[0].start
        end = region[0].stop
        region_signal = SWR_sum[start:end]
        peak_relative = np.argmax(region_signal)
        peak_sample = start + peak_relative
        peak_time_sec = peak_sample / eeg_fs
        peak_value = SWR_sum[peak_sample]

        region_peaks.append({
            "start_sample": start,
            "end_sample": end,
            "peak_sample": peak_sample,
            "peak_time_sec": peak_time_sec,
            "peak_value": SWR_sum[peak_sample],
            "duration_ms": (end - start) / eeg_fs * 1000
        })

    print(f"{patient_id}: Found {len(region_peaks)} SWR events.")
    df_regions = pd.DataFrame(region_peaks)
    df_regions.to_csv(op.join(region_output_path, f"{patient_id}_SWR_regions.csv"), index=False)\
    
    #regions of no SWRs    
    valid_no_regions = (SWR_sum == 0)
    #dialate and erode to merge neighbouring regions
    min_duration_samples = int(10)
    structure_element = np.ones(min_duration_samples)
    valid_no_regions = binary_dilation(valid_no_regions, structure=structure_element)
    valid_no_regions = binary_erosion(valid_no_regions, structure=structure_element)
    
    labeled_low_regions, num_low_regions = label(valid_no_regions)
    low_slices = find_objects(labeled_low_regions)

    low_regions = []
    for low_region in low_slices:
        start = low_region[0].start
        end = low_region[0].stop
        duration_ms = (end - start) / eeg_fs * 1000
        midpoint_sample = (start + end) // 2 

        low_regions.append({
        "start_sample": start,
        "end_sample": end,
        "midpoint_sample": midpoint_sample,
        "duration_ms": duration_ms})


        
    print(f"{patient_id}: Found {len(low_regions)} no SWR events.")


    #2. make the MNE epochs

    #improt raw
    raw = mne.io.read_raw_fif(raw_files[patient_id], preload = True, verbose = False)
    #notch filter
    notch_freqs = [50, 100, 150]
    raw = raw.notch_filter(freqs=notch_freqs, notch_widths=4, verbose = False)
    raw_selected = raw.copy()
    data = raw_selected.get_data()
    info = raw_selected.info
    data_elect = mne.io.RawArray(data, info)

    #events array for SWRs
    if len(df_regions) < no_of_epochs:
        print(f"{patient_id}: Less than {no_of_epochs} SWRs regions detected. Using all {len(df_regions)} available regions.")
    else:
        df_regions = df_regions.sort_values("peak_value", ascending = False).head(no_of_epochs)

    df_regions = df_regions.sort_values("peak_sample") #to enusre chronological order
    events = []
    last_event = -min_gap
    used_swr_regions = 0
    for row in df_regions.itertuples():
        peak_sample = row.peak_sample
        if peak_sample >= last_event + min_gap:
            events.append([peak_sample, 0, 1])
            last_event = peak_sample
            used_swr_regions += 1

    if used_swr_regions < no_of_epochs:
     print(f"{patient_id}: Only {used_swr_regions} SWR epochs could be used after applying min_gap constraint.")
    
    #events for no SWRs
    if len(low_regions) < no_of_epochs:
        print(f"{patient_id}: Less than {no_of_epochs} low-SWR regions detected. Using all {len(low_regions)} available regions.")
        random_low_regions = low_regions
    else:   
        random_low_regions = random.sample(low_regions, no_of_epochs)
        random_low_regions = sorted(random_low_regions, key=lambda x: x["midpoint_sample"]) 

    for region in random_low_regions:
        midpoint_sample = region["midpoint_sample"]
        events.append([midpoint_sample, 0, 2])
    
    events = np.array(events, dtype=int)
    if len(events) == 0:
       print(f"{patient_id}: No valid events found for epoching.")
       continue

    high_epochs = mne.Epochs(data_elect, events, event_id=1, tmin=-1, tmax=1, picks = ['grad', 'mag'], preload=True, verbose = False)
    low_epochs = mne.Epochs(data_elect, events, event_id=2, tmin=-1, tmax=1, picks = ['grad', 'mag'], preload=True, verbose = False)
    all_epochs = mne.Epochs(data_elect, events, event_id= [1, 2], tmin=-1, tmax=1, picks = ['grad', 'mag'], preload=True, verbose = False)
   
   
    #save
    high_epoch_filename = op.join(epochs_save_path, f"{patient_id}_high_SWR_epochs.fif")
    low_epoch_filename = op.join(epochs_save_path, f"{patient_id}_low_SWR_epochs.fif")
    all_epoch_filename = op.join(epochs_save_path, f"{patient_id}_all_SWR_epochs.fif")
    high_epochs.save(high_epoch_filename, overwrite=True)
    low_epochs.save(low_epoch_filename, overwrite=True)
    all_epochs.save(all_epoch_filename, overwrite=True)

    print(len(all_epochs))
    print(len(high_epochs))
    print(len(low_epochs))

    print(f"{patient_id}: Drop log for high SWR epochs:")
    print(high_epochs.drop_log)

    print(f"{patient_id}: Drop log for low SWR epochs:")
    print(low_epochs.drop_log)

    print(f"{patient_id}: Drop log for all epochs:")
    print(all_epochs.drop_log)
    #print(f"{patient_id}: Saved {len(epochs)} epochs to {epoch_filename}")



Patient5_part1L: Found 94 SWR events.
Patient5_part1L: Found 95 no SWR events.


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_65405/564586550.py:89: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WZ_patient5/WZ_LHippo_001_tsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_files[patient_id], preload = True, verbose = False)


Creating RawArray with float64 data, n_channels=333, n_times=358000
    Range : 0 ... 357999 =      0.000 ...   357.999 secs
Ready.
Overwriting existing file.


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_65405/564586550.py:137: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = mne.Epochs(data_elect, events, event_id= [1, 2], tmin=-3, tmax=3, picks = ['grad', 'mag'], preload=True, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_65405/564586550.py:144: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_epochs/theta/Patient5_part1L_high_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  high_epochs.save(high_epoch_filename, overwrite=True)


Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.


...

60
30
30
Patient23: Drop log for high SWR epochs:
((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',))
Patient23: Drop log for low SWR epochs:
(('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IG

/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_65405/564586550.py:89: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/meg/WL_patient25/WL_raw_rest_001_tsss.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_files[patient_id], preload = True, verbose = False)


Creating RawArray with float64 data, n_channels=386, n_times=256000
    Range : 0 ... 255999 =      0.000 ...   255.999 secs
Ready.


/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_65405/564586550.py:137: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = mne.Epochs(data_elect, events, event_id= [1, 2], tmin=-3, tmax=3, picks = ['grad', 'mag'], preload=True, verbose = False)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_65405/564586550.py:144: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_epochs/theta/Patient25_high_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  high_epochs.save(high_epoch_filename, overwrite=True)
/var/folders/hp/k3clndsn2sj608qsb4fl5zvr0000gn/T/ipykernel_65405/564586550.py:145: RuntimeWarning: This filename (/Users/Lea/Desktop/PROJECT_DATA_ANALYSIS/MEG_epochs/theta/Patient25_low_theta_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz

59
30
29
Patient25: Drop log for high SWR epochs:
((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',))
Patient25: Drop log for low SWR epochs:
(('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IGNORED',), ('IG

In [68]:
for patient_id, electrodes in hippocampus_electrodes.items():
    df = pd.read_csv(detected_SWRs[patient_id])
    df = df[df['Channel'].isin(electrodes)]
    if df.empty:
        print(f"No hippocampal channels found for {patient_id}")
        continue
    SWR_times = (df['rippleTime'].apply(lambda x: eval(x, {'array': np.array}))).tolist()
    duration = recording_durations[patient_id]
    eeg_no_samples = int(duration * eeg_fs)
    no_channels = len(df)
    SWR_array = np.zeros((no_channels, eeg_no_samples), dtype=int)
    for channel, times in enumerate(SWR_times):
        if not times or all(len(t) == 0 for t in times):
            ripple_time_value = df.iloc[channel]['rippleTime'] 
            print(f"Skipping empty SWR_times for channel index {channel} in patient {patient_id}")
            continue
        channel_SWR_times = np.hstack(times)
        channel_SWR_idx = np.round((channel_SWR_times * eeg_fs), 0).astype(int)
        SWR_array[channel, channel_SWR_idx] = 1

    SWR_sum = np.sum(SWR_array, axis = 0)

    #find regions where mutliple SWRs across channels detected == sum > 3
    SWRs_region = (SWR_sum >= (0.25*len(electrodes)))
    SWR_regions = np.diff(SWRs_region, prepend = 0, append = 0)
    start_SWR = np.where(SWR_regions == 1)[0]
    end_SWR = np.where(SWR_regions == -1)[0]

    SWRs = [(int(start), int(end)) for start, end in zip(start_SWR,end_SWR ) if 10 <= (end - start)  ] # <= 100 ]
    #same for no SWRs
    print(f"{patient_id}: Found {len(SWRs)} SWR events.")


No hippocampal channels found for Patient3_part1
Patient5_part1L: Found 77 SWR events.
Patient6: Found 157 SWR events.
Patient8: Found 333 SWR events.
No hippocampal channels found for Patient9
Patient10: Found 94 SWR events.
Patient11_part1: Found 227 SWR events.
No hippocampal channels found for Patient12_part1
Patient13: Found 79 SWR events.
Patient14_part1: Found 169 SWR events.
No hippocampal channels found for Patient15
Patient16: Found 37 SWR events.
Patient17: Found 141 SWR events.
Patient21: Found 119 SWR events.
Patient22: Found 33 SWR events.
Patient23: Found 449 SWR events.
Patient25: Found 22 SWR events.
